In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtester_rt import *
import matplotlib.pyplot as plt
from time import time
from passivbot import load_settings

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)

In [ ]:
settings = load_settings('default')
all_coins = set(settings['coins'])
settings

In [ ]:
p = 1.529
max_span = 60 * 18
n = 11
spans = np.array([p**i for i in range(n)])
spans = list((spans / spans.max() * max_span).round().astype(int))
spans

In [ ]:
# modify loaded settings
for c in settings['coins']:
    settings['coins'][c]['profit_pct'] = 0.005
    settings['coins'][c]['account_equity_pct_per_trade'] = 0.001
    settings['coins'][c]['entry_spread'] = 0.001
    settings['coins'][c]['account_equity_pct_per_hour'] = 0.0005
settings['ema_spans_minutes'] = [15, 1080] # used only smallest and largest ema to speed up ema calculation
                                           # comparing min_emas([15, 1080]) to
                                           # min_emas([15, 25, 40, 64, 102, 164, 263, 421, 675, 1080])
                                           # shows negligible difference
            
            
            
            
            
# necessary backtester settings
settings['start_quot'] = 0.15
settings['fee'] = 0.00075 # vip1
settings['margin'] = 5
if settings['margin'] == 5:
    settings['liquidation_margin_level'] = 1.05
elif settings['margin'] == 3:
    settings['liquidation_margin_level'] = 1.1
else:
    settings['liquidation_margin_level'] = 1.1
settings['min_quot_cost'] = 0.0001
settings

In [ ]:
symbols = [c + '/BTC' for c in all_coins]
symbols = sorted(symbols)

n_days = 30 * 3
symbols, len(symbols)

In [ ]:
df = create_df(symbols, n_days, settings, no_download=False) # set no_download to True to use cached raw_trades

In [ ]:
bl, entrl, exitl, exitpl = backtest(df, settings)

In [ ]:
bldf = pd.DataFrame(bl).set_index('timestamp')

In [ ]:
start_equity = bldf.acc_equity_quot.iloc[0]
end_equity = bldf.acc_equity_quot.iloc[-1]
n_days = (bldf.index[-1] - bldf.index[0]) / 1000 / 60 / 60 / 24
avg_daily_gain = (end_equity / start_equity)**(1 / n_days)
hwme = bldf.acc_equity_quot.max()
hwmi = bldf.acc_equity_quot.idxmax()
n_days_hwm = (hwmi - bldf.index[0]) / 1000 / 60 / 60 / 24
max_daily_gain = (hwme / start_equity)**(1 / n_days_hwm)
plt.plot(np.linspace(0, 100, len(bldf)), bldf.acc_equity_quot.values)


print('               ndays', round(n_days, 2))
print('start end daily gain', round(avg_daily_gain, 8))
print('      max daily gain', max_daily_gain)
print('      low water mark', bldf.acc_equity_quot.min())
print('     high water mark', hwme)
print('                 end', bldf.acc_equity_quot.iloc[-1])
print('                mean', bldf.acc_equity_quot.mean())
print('         yearly gain', max_daily_gain ** 365)

xs = np.linspace(0, 100, len(bldf))
plt.plot(xs, bldf.acc_equity_quot.values)
wema = bldf.acc_equity_quot.ewm(span=100).mean().values
plt.plot(xs, wema)
print('            ema gain', (wema[-1] / wema[0])**(1 / n_days))



In [ ]:
margin_level = (bldf.onhand_ito_quot / bldf.acc_debt_quot).replace([np.inf, -np.inf], np.nan)
margin_level = margin_level[margin_level < 2]
margin_level = margin_level[margin_level > 0]
if len(margin_level) > 1:
    print('margin level')
    plt.plot([margin_level.index[0], margin_level.index[-1]], [settings['liquidation_margin_level']] * 2)
    margin_level.plot()


In [ ]:
bldfc = bldf.drop(['acc_equity_quot', 'acc_debt_quot', 'BTC', 'onhand_ito_quot', 'credit_avbl_quot'], axis=1)
bldfc.plot()
print('\nbiggest shrt bags')
print(bldfc.min().sort_values())
print('\nbiggest long bags')
print(bldfc.max().sort_values()[::-1])

In [ ]:
print('sum_bag_size / acc_equity green')
print('credit_avbl_quot purple')
(bldfc.abs().sum(axis=1) / bldf.acc_equity_quot).plot(style='g-')
bldf.credit_avbl_quot.plot(style='m-')

In [ ]:
tdfs = {}
analyses = {}
for s in symbols:
    print(s)
    entrdf = pd.DataFrame([e for e in entrl if e['symbol'] == s]).set_index('timestamp')
    exitdf = pd.DataFrame([e for e in exitl if e['symbol'] == s]).set_index('timestamp')
    pricedf = pd.DataFrame([e for e in exitpl if e['symbol'] == s]).set_index('timestamp')
    tdfs[s] = {
        'lentr': entrdf[entrdf.side == 'buy'],
        'sentr': entrdf[entrdf.side == 'sell'],
        'lexit': exitdf[exitdf.side == 'sell'],
        'sexit': exitdf[exitdf.side == 'buy'],
        'lprice': pricedf[pricedf.side == 'sell'],
        'sprice': pricedf[pricedf.side == 'buy'],
        'buys': pd.DataFrame([e for e in entrl + exitl if e['symbol'] == s and e['side'] == 'buy']).set_index('timestamp'),
        'sels': pd.DataFrame([e for e in entrl + exitl if e['symbol'] == s and e['side'] == 'sell']).set_index('timestamp')
        
    }
    levwap = tdfs[s]['lentr'].cost.sum() / tdfs[s]['lentr'].amount.sum()
    sevwap = tdfs[s]['sentr'].cost.sum() / tdfs[s]['sentr'].amount.sum()
    analyses[s] = {'levwap': levwap, 'sevwap': sevwap, 'sele': sevwap / levwap}

In [ ]:
s = 'ETH/BTC'
lentr = tdfs[s]['lentr']
sentr = tdfs[s]['sentr']
lexit = tdfs[s]['lexit']
sexit = tdfs[s]['sexit']
lprice = tdfs[s]['lprice']
sprice = tdfs[s]['sprice']

In [ ]:
lprice.price.plot()
sprice.price.plot()

In [ ]:
lentr.price.plot(style='b.-')
sentr.price.plot(style='r.-')
levwap = lentr.cost.sum() / lentr.amount.sum()
sevwap = sentr.cost.sum() / sentr.amount.sum()
levwap, sevwap, sevwap / levwap

In [ ]:
lprice.price.plot()
lentr.price.plot(style='b.-')
lexit.price.plot(style='ro')
lxvwap = lexit.cost.sum() / lexit.amount.sum()
levwap, lxvwap, lxvwap / levwap

In [ ]:
sprice.price.plot()
sentr.price.plot(style='r.-')
sexit.price.plot(style='bo')
sxvwap = sexit.cost.sum() / sexit.amount.sum()
sevwap, sxvwap, sevwap / sxvwap

In [ ]:
lexit.price.plot(style='ro-')
sexit.price.plot(style='bo-')
lxvwap, sxvwap, lxvwap / sxvwap

In [ ]:
bvwap = tdfs[s]['buys'].cost.sum() / tdfs[s]['buys'].amount.sum()
svwap = tdfs[s]['sels'].cost.sum() / tdfs[s]['sels'].amount.sum()
bvwap, svwap, svwap / bvwap